Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [19]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from tqdm import trange


## The *Nim* and *Nimply* classes

In [20]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [21]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [22]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [23]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [24]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [25]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [26]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=5)
INFO:root:status: <0 3 5 7 4>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 3 1 7 4>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 1 7 4>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=3)
INFO:root:status: <0 2 1 4 4>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 1 1 4 4>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=2)
INFO:root:status: <0 1 1 2 4>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=2)
INFO:root:status: <0 1 1 0 4>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0 1 0 4>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=3)
INFO:root:status: <0 0 1 0 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0

## Evolution Strategies

### Utils

In [27]:
class Individual:
    genotype:float  # a number between 0 and 1, it's the probability of choosing the move with the smallest non-zero nim sum at any given time
    fitness: tuple

    def __init__(self, genotype) -> None:
        self.genotype = genotype

    def __str__(self) -> str:
        return f"{self.genotype}: {self.fitness}"

    def phenotype(self, state: Nim) -> Nimply:
        analysis = analize(state)
        spicy_moves = [(ply, ns) for ply, ns in analysis["possible_moves"].items() if ns != 0]
        if not spicy_moves:
            spicy_moves = list(analysis["possible_moves"].keys())
            return random.choice(spicy_moves)
        if random.random() < self.genotype: # choose the move with the smallest non-zero nim sum
            return min(spicy_moves, key = lambda i: i[1])[0]
        return random.choice(spicy_moves)[0]

In [28]:
def game(player1, player2=pure_random) -> (bool, int):  # plays a game against an opponent who randomly picks their move
    nim = Nim(5)
    strategy = (player1, player2)
    player = random.choice((0,1))  # randomly select starting player
    num_moves = 0
    while nim:
        num_moves += 1
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = not player
    return not player, num_moves  # returns the losing player (1 if the winner is 0 and viceversa) and the duration of the game

In [29]:
def fitness(individual:Individual):  # the fitness is calculated as the winrate and the average number of moves calculated over 100 games
    winrate = 0
    tot_moves = 0.0
    for _ in range(100):
        won, moves = game(individual.phenotype)
        winrate += won
        tot_moves += moves
    return winrate, - tot_moves/100

In [30]:
ind = Individual(random.random())

ind.fitness = fitness(ind)
print(ind.genotype, ind.fitness)

0.6417509636735369 (89, -10.11)


In [31]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 10
PARENTS_PER_CHILD = 2
TOUR_SIZE = 2
MUTATION_CHANCE = 0.05
SIGMA = 0.2
TOTAL_GENERATIONS = 100

In [32]:
def mutate(ind: Individual):
    return Individual(np.clip(np.random.normal(ind.genotype, SIGMA), 0, 1))

In [33]:
def xover(individuals):
    return Individual(np.average([i.genotype for i in individuals]))

In [34]:
def tour(individuals):
    tournament = [random.choice(individuals) for _ in range(TOUR_SIZE)]
    champion = max(tournament, key=lambda i: i.fitness)
    return champion

In [38]:
# initialize the population
def init_pop():
    population = [Individual(random.random()) for _ in range(POPULATION_SIZE)]
    for p in population:
        p.fitness = fitness(p)
    return population


population = init_pop()

## ($\mu$ + $\lambda$) ES

In [36]:
for generation in range(TOTAL_GENERATIONS):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        if random.random() < MUTATION_CHANCE: # mutate
            parent = tour(population)
            child = mutate(parent)
        else:
            parents = list()
            for _ in range(PARENTS_PER_CHILD):
                parents.append(tour(population))
            child = xover(parents)
        child.fitness = fitness(child)
        offspring.append(child)
    population.extend(offspring)
    population.sort(key = lambda i: i.fitness, reverse=True)
    population = population[: POPULATION_SIZE]
    print(population[0])
        


0.9760328053354482: (96, -9.75)
0.9760328053354482: (96, -9.75)
0.9760328053354482: (96, -9.75)
0.9760328053354482: (96, -9.75)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
1.0: (98, -9.85)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.8971933258631208: (98, -9.8)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.9384271856757413: (99, -9.97)
0.93842

## ($\mu$, $\lambda$) ES

In [39]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 30
PARENTS_PER_CHILD = 2
TOUR_SIZE = 2
MUTATION_CHANCE = 0.05
SIGMA = 0.2
TOTAL_GENERATIONS = 100

population = init_pop()

In [40]:
for generation in range(TOTAL_GENERATIONS):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        if random.random() < MUTATION_CHANCE: # mutate
            parent = tour(population)
            child = mutate(parent)
        else:
            parents = list()
            for _ in range(PARENTS_PER_CHILD):
                parents.append(tour(population))
            child = xover(parents)
        child.fitness = fitness(child)
        offspring.append(child)
    offspring.sort(key = lambda i: i.fitness, reverse=True)
    population = offspring[: POPULATION_SIZE]
    print(population[0])

0.8680526670492053: (96, -10.0)
0.8680526670492053: (97, -9.85)
0.8680526670492053: (98, -10.11)
0.9424098417244118: (95, -9.88)
0.9340263335246026: (96, -9.78)
0.9392532359955659: (97, -9.47)
0.9243381222411395: (98, -9.87)
0.921194306666211: (96, -9.82)
0.921194306666211: (98, -9.61)
0.9073222776312692: (96, -9.96)
0.9140380052818605: (96, -10.04)
0.9211543340794667: (97, -9.99)
0.9268283504935139: (97, -9.58)
0.9317972240097103: (96, -9.66)
0.9317746847246935: (96, -9.97)
0.94669755856185: (95, -9.9)
0.9535801576740937: (98, -9.54)
0.9544667326645633: (96, -9.71)
0.9560588346415241: (96, -10.03)
0.9703554840066456: (98, -10.01)
0.965564131850255: (98, -9.67)
0.9643141661751281: (97, -9.77)
0.9643141661751281: (97, -9.96)
0.9618638599751751: (95, -9.8)
0.9624615275991428: (95, -9.62)
0.9633881956544306: (97, -9.65)
0.9631366826567739: (97, -9.75)
0.9630581597804491: (96, -9.82)
0.9630581597804491: (96, -9.61)
0.9628165479767539: (97, -9.95)
1.0: (97, -10.24)
0.9813002300979874: (97, 